# Receiver with Spark
We will use this notebook as a receiver to receive the data from the sender notebook. We use the same port number as the sender notebook.

In theory, We have to run the sender notebook before running this notebook.

However, we cannot run this receiver with sparkstreaming because we cannot make sparkstrateam work. We got trouble in spark envrionment setup on our laptops.

## Envrionment

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from textblob import TextBlob
import socket

## Rceivng the data

In [2]:
# create a spark context
sc = SparkContext(appName="TwitterSentimentAnalysis")
sc.setLogLevel("ERROR")

# create a local StreamingContext with two working thread and batch interval of 1 second
ssc = StreamingContext(sc, 3)

# create a DStream that will connect to hostname:port, like localhost:8080
host = socket.gethostname()
port = 8080
lines = ssc.socketTextStream(host, port)

print("Connection established with the server...")

23/05/02 18:00:41 WARN Utils: Your hostname, ZheHUANGs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.3 instead (on interface en0)
23/05/02 18:00:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/02 18:00:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


Connection established with the server...


## Sentiment analysis

In [3]:
# define a function to perform sentiment analysis on each message
def analyze_sentiment(message):
    blob = TextBlob(message)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        return "positive"
    elif sentiment < 0:
        return "negative"
    else:
        return "neutral"

In [4]:
# tag each message with its sentiment
sentiments = lines.map(lambda message: (message, analyze_sentiment(message)))

# print the tagged messages
sentiments.pprint()

# start the streaming computation
ssc.start()

# wait for the streaming to finish
ssc.awaitTermination()

-------------------------------------------
Time: 2023-05-02 18:00:45
-------------------------------------------



objc[91005]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called.
objc[91005]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead. Set a breakpoint on objc_initializeAfterForkError to debug.
23/05/02 18:00:48 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 810, in main
    eval_type = read_int(infile)
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner

Py4JJavaError: An error occurred while calling o24.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/streaming/dstream.py", line 254, in takeAndPrint
    taken = rdd.take(num + 1)
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/rdd.py", line 2836, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/context.py", line 2319, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/py4j/java_gateway.py", line 1322, in __call__
    return_va

-------------------------------------------
Time: 2023-05-02 18:00:51
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:00:54
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:00:57
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:03
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:06
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:09
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:12
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:15
----------

23/05/02 18:01:15 ERROR ReceiverTracker: Deregistered receiver for stream 0: Restarting receiver with delay 2000ms: Socket data stream had no more data
objc[91009]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called.
objc[91009]: +[__NSCFConstantString initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead. Set a breakpoint on objc_initializeAfterForkError to debug.
23/05/02 18:01:18 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 810, in main
    eval_type = read_int(infile)
  File "/Users/zhehuang/opt/anaconda3/envs/Spark/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py

-------------------------------------------
Time: 2023-05-02 18:01:21
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:24
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:27
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:30
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:33
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:36
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:39
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:42
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:45
----------

-------------------------------------------
Time: 2023-05-02 18:01:51
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:54
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:01:57
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:03
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:06
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:09
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:12
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:15
----------

-------------------------------------------
Time: 2023-05-02 18:02:51
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:54
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:02:57
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:03:00
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:03:03
-------------------------------------------

-------------------------------------------
Time: 2023-05-02 18:03:06
-------------------------------------------

